In [5]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [7]:
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [8]:

import time

guidance_scale = 15.0
prompt = ["a barbie doll"] * 1
batch_size = len(prompt)

print(f"Prompt: {prompt}")

start_time = time.time()
latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=prompt),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=32,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

print(f"Elapsed diffusion: {(time.time() - start_time)}")

from shap_e.util.notebooks import decode_latent_mesh
p = decode_latent_mesh(xm, latents[0])
print(p.verts.size())
print(p.faces.size())
print(p.vertex_channels["R"].size())
print(p.vertex_channels["G"].size())
print(p.vertex_channels["B"].size())
print(p.face_channels)
exit()

Prompt: ['a barbie doll']


  0%|          | 0/32 [00:00<?, ?it/s]

Elapsed diffusion: 4.250999927520752
torch.Size([11232, 3])
torch.Size([22444, 3])
torch.Size([11232])
torch.Size([11232])
torch.Size([11232])
{}


In [9]:
render_mode = 'nerf' # you can change this to 'stf'
size = 64 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    start_time = time.time()
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    print(f"Elapsed decode-{i}: {(time.time() - start_time)}")
    display(gif_widget(images))

Elapsed decode-0: 2.1469991207122803


HTML(value='<img src="data:image/gif;base64,R0lGODlhQABAAIcAAPjT4/nR4/jQ4vDLzurM0efL0ObMzu/K2fLJzvLJze/KzO7H1P…

In [10]:
# Example of saving the latents as meshes.
from shap_e.util.notebooks import decode_latent_mesh

for i, latent in enumerate(latents):
    t = decode_latent_mesh(xm, latent).tri_mesh()
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        t.write_ply(f)
    with open(f'example_mesh_{i}.obj', 'w') as f:
        t.write_obj(f)